In [1]:
'''
In this notebook, we train take a sequence as an input, encode it with a deep autoencoder, and 
output the element-wise encoding error. This output can be used for calculating a metrics for anomaly detection.
'''
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

## Data definition

In [2]:
def get_normal_reading(dim, noise_scale):
    w = np.linspace(2.6, 5.8, dim) # frequency
    s = np.linspace(1.2, 9.4, dim)  # phase
    
    t = np.linspace(0, 10.0, 501)
    reading = []
    for i in range(dim):
        signal = np.sin(w[i]*t + s[i])
        noise = np.random.normal(0, noise_scale, len(t))
        reading.append( signal + noise )
    reading = np.array(reading)
    return reading


def get_normal_data(dim, sequence_length, normal_sample_size, noise_scale, **kwargs):
    '''
    x contains the sequences draw from the normal readings. x has the shape of (normal_sample_size, sequence_length, dim)
    y contains the label of if the sequence is an anomaly. In this case, all labels in y are 0.
    '''
    reading = get_normal_reading(dim, noise_scale)
      
    x = []
    y = []
    for _ in range(normal_sample_size):
        i = np.random.randint(0, len(reading[0])-sequence_length-1)
        x.append(reading[:, i:i+sequence_length])
        y.append(0)
        
    x = np.asarray(x, dtype=np.float32)
    # change the shape of x from (normal_sample_size, dim, sequence_length) to (normal_sample_size, sequence_length, dim)
    x = x.transpose((0, 2, 1))
    
    y = np.asarray(y, dtype=np.float32)
    
    return x, y


from scipy import signal
def get_mixed_data(dim, sequence_length, mixed_sample_size, 
                   noise_scale, abnormal_ratio, anomaly_magnitude, **kwargs):
    '''
    x contains the sequences draw from the normal/abnormal readings. 
    x has the shape of (mixed_sample_size, sequence_length, dim)
    
    y contains the label of if the sequence is an anomaly. y=1 means the sequence is abnormal.
    '''
    window_size = 13 # anomaly windows size
    window_std = 2 # anomaly standard deviation
    anomaly_dim = 0
    
    x, y = get_normal_data(dim, sequence_length, mixed_sample_size, noise_scale)
    for i in range(mixed_sample_size):
        isAnomaly = np.random.choice([0, 1], p=[1-abnormal_ratio, abnormal_ratio])
        if isAnomaly == 1:
            anomaly = anomaly_magnitude * signal.gaussian(window_size, std=window_std)
            idx = np.random.randint(0, sequence_length-window_size)
            x[i, idx:idx+window_size, anomaly_dim] += anomaly
            y[i] = 1
    
    return x, y

## Data generation

In [3]:
params = {'dim':1, 'sequence_length':37, 'noise_scale':0.05, 
          'normal_sample_size':8192, 'mixed_sample_size':2048, 'abnormal_ratio':0.5, 'anomaly_magnitude':0.5}

np.random.seed(9527)
x_train, y_train = get_normal_data(**params)
x_test, y_test = get_mixed_data(**params)

zero_train = np.zeros((x_train.shape[0], x_train.shape[1], x_train.shape[2]))
zero_test = np.zeros((x_test.shape[0], x_test.shape[1], x_test.shape[2]))

## FC model

In [4]:
import tensorflow as tf

def get_tf_model_fc(dim, sequence_length):
    input_sequence = tf.keras.layers.Input(shape=(dim*sequence_length, ), name='input_sequence')
    x = tf.keras.layers.Dense(units=128, activation='relu')(input_sequence)
    decoded = tf.keras.layers.Dense(units=dim*sequence_length)(x)

    diff = tf.keras.layers.Subtract(name='encoding_error')([input_sequence, decoded])
    model = tf.keras.Model(input_sequence, diff)
    model.compile(optimizer='adam', loss='mse')
    
    return model

/data/anaconda/envs/deeplearning/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/anaconda/envs/deeplearning/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/anaconda/envs/deeplearning/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/anaconda/envs/deeplearning/lib/python3.7/site-

In [5]:
x_train_flatten = np.reshape(x_train, (x_train.shape[0], np.prod(x_train.shape[1:])))
x_test_flatten = np.reshape(x_test, (x_test.shape[0], np.prod(x_test.shape[1:])))

zero_train_flatten = np.reshape(zero_train, (zero_train.shape[0], np.prod(zero_train.shape[1:])))
zero_test_flatten = np.reshape(zero_test, (zero_test.shape[0], np.prod(zero_test.shape[1:])))

model = get_tf_model_fc(dim=params['dim'], sequence_length=params['sequence_length'])

model.fit(x_train_flatten, zero_train_flatten, batch_size=64, epochs=20, 
          validation_data=(x_test_flatten[y_test==0], zero_test_flatten[y_test==0]))

pred_origin_model = model.predict(x_test_flatten)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 8192 samples, validate on 1039 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
8192/8192 [==============================] - 1s 157us/sample - loss: 0.0611 - val_loss: 0.0035
Epoch 2/20
8192/8192 [==============================] - 0s 33us/sample - loss: 0.0032 - val_loss: 0.0033
Epoch 3/20
8192/8192 [==============================] - 0s 33us/sample - loss: 0.0030 - val_loss: 0.0031
Epoch 4/20
8192/8192 [==============================] - 0s 33us/sample - loss: 0.0028 - val_loss: 0.0029
Epoch 5/20
8192/8192 [==============================] - 0s 33us/sample - loss: 0.0027 - val_loss: 0.0028
Epoch 6/20
8192/8192 [==============================] - 0s 38us/sample - loss: 0.0025 - val_loss: 0.0027
Epoch 7/20
8192/8192 [==============================] - 0s 33us/sample - loss: 0.0024 - val_loss: 0.0026
Epoch 8/20
8192/8192 [==========================

## Output the model to a .pb file

In [6]:
# reference:
# https://stackoverflow.com/questions/45466020/how-to-export-keras-h5-to-tensorflow-pb
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.compat.v1.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.compat.v1.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [7]:
import tensorflow as tf

sess = tf.compat.v1.keras.backend.get_session()
frozen_graph = freeze_session(sess, output_names=[out.op.name for out in model.outputs])

output_tf_model = 'Anomaly_detection_model_v2.pb'
# Finally we serialize and dump the output graph to the filesystem
tf.io.write_graph(frozen_graph, "./", output_tf_model, as_text=False)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 21 variables.
INFO:tensorflow:Converted 21 variables to const ops.


'./Anomaly_detection_model_v2.pb'

## Load the model and use it in TensorFlow
Reference: https://leimao.github.io/blog/Save-Load-Inference-From-TF-Frozen-Graph/

In [8]:
tf.reset_default_graph()

graph = tf.Graph()
with tf.gfile.GFile('./' + output_tf_model, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

In [9]:
nodes = [n.name + ' => ' +  n.op for n in graph_def.node if n.op in ('Placeholder')]
for node in nodes:
    print(node)

input_sequence => Placeholder


In [10]:
with graph.as_default():
    # Define input tensor
    input_tensor = tf.placeholder(np.float32, shape = [None, params['sequence_length']*params['dim']])
    tf.import_graph_def(graph_def, {'input_sequence': input_tensor})
graph.finalize()

In [11]:
layers = [op.name for op in graph.get_operations()]
for layer in layers:
    print(layer)

Placeholder
import/input_sequence
import/dense/kernel
import/dense/bias
import/dense/MatMul/ReadVariableOp
import/dense/MatMul
import/dense/BiasAdd/ReadVariableOp
import/dense/BiasAdd
import/dense/Relu
import/dense_1/kernel
import/dense_1/bias
import/dense_1/MatMul/ReadVariableOp
import/dense_1/MatMul
import/dense_1/BiasAdd/ReadVariableOp
import/dense_1/BiasAdd
import/encoding_error/sub
import/Adam/iterations
import/Adam/lr
import/Adam/beta_1
import/Adam/beta_2
import/Adam/decay
import/training/Adam/Variable
import/training/Adam/Variable_1
import/training/Adam/Variable_2
import/training/Adam/Variable_3
import/training/Adam/Variable_4
import/training/Adam/Variable_5
import/training/Adam/Variable_6
import/training/Adam/Variable_7
import/training/Adam/Variable_8
import/training/Adam/Variable_9
import/training/Adam/Variable_10
import/training/Adam/Variable_11


In [12]:
sess = tf.Session(graph=graph)
output_tensor = graph.get_tensor_by_name("import/encoding_error/sub:0")
pred_loaded_model = sess.run(output_tensor, feed_dict={input_tensor: x_test_flatten})

# The difference should be very close to 0
np.sum(np.abs(pred_loaded_model - pred_origin_model))

0.0